In [ ]:
from functools import reduce
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
import warnings
warnings.filterwarnings("ignore")

pd.set_option("display.max_columns", None)

In [ ]:
# Initialize to avoid load error, set
# actual values at execution step.
YEARS = []  
OFFICES = []


''' Remove top features not shared between 
    different datasets to prevent errors.'''
def removeUncommonColumns(nested_dict):
    print("Removing uncommon columns...")
    
    # Flatten and find common columns
    all_dfs = [df for year in nested_dict for df in nested_dict[year].values()]
    common_cols = set(all_dfs[0].columns)
    for df in all_dfs[1:]:
        common_cols &= set(df.columns)
    
    # Safely trim all dataframes
    for year in nested_dict:
        for office in nested_dict[year]:
            df = nested_dict[year][office]
            existing_cols = [col for col in common_cols if col in df.columns]
            nested_dict[year][office] = df[existing_cols]

    print('Done.')
    
    return nested_dict


''' Pull the engineered feature data along with its
    target for each year and office.'''
def makeDatasets(years, offices):
    print('Making datasets...')
    
    df_datasets = {}
    
    for year in years:
        print(f'Processing year {year}...')
        df_datasets[year] = {}
        
        for office in offices:
            print(f'Processing office {office}...')

            df = pd.read_csv('data/generated_data/07_ml_features_' + year + '_' + office + '.csv', low_memory=False)

            # for col in df.columns:
                # if col.startswith(('S', 'B', 'D')):
                    # df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0.0)
                    # invalid_values = ['(X)', '-', 'N/A', 'null', '']
                    # df[col] = df[col].replace(invalid_values, np.nan)

            
            # For running this function separately
            # on other years and offices outside of
            # the main execution loop.
            # For example, 2024 has columns that other
            # years do not, which causes errors.
            # if remove_xtra_cols:
                # df = df.drop(columns=remove_xtra_cols, errors='ignore')

            df_datasets[year][office] = df
    
    df_datasets = removeUncommonColumns(df_datasets)

    print('Done.')
    
    return df_datasets


''' Split features and target into X
    and y variables with some cleanup.'''
def makeFeaturesTargets(df):
    print(f'Making features and target...')
    
    y = df[['partisan_temp']]
    X = df.drop(columns=['standardized_id_num', 'partisan_temp', 'partisan_temp_change_curr'])
    X = X.replace(['-', '(X)', 'N/A', 'null', ''], pd.NA)

    print('Done.')
    return X, y


''' Pipeline to impute and encode categorical
    variables, as well as scale, etc.'''
def fitModel(X, y):
    print(f'Fitting model...')

    categorical_cols = [
        'office_code',
        'partisanship_lean_curr',
        'partisanship_lean_prev',
        'partisanship_lean_change_prev',
    ]

    categorical_cols = [col for col in categorical_cols if col in X.columns]
    numeric_cols = X.select_dtypes(include=['number']).columns.tolist()

    numeric_cols = [col for col in numeric_cols if col not in categorical_cols]

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OneHotEncoder(drop='first', handle_unknown='ignore'))
    ])

    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ])

    preprocessor = ColumnTransformer(transformers=[
        ('cat', categorical_transformer, categorical_cols),
        ('num', numeric_transformer, numeric_cols)
    ])

    model = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', LinearRegression())
    ])
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model.fit(X_train, y_train)

    print('Done.')

    return model, X_train, X_test, y_train, y_test, numeric_cols


def makePredictions(X_test, model):
    print(f'Making predictions...')
    y_pred = model.predict(X_test)
    print('Done.')
    return y_pred


def plotAccuracy(y_test, y_pred):
    print(f'Plotting accuracy...')
    plt.figure(figsize=(12, 9))
    plt.scatter(y_test, y_pred)
    plt.xlim(-1.25, 1.25)
    plt.ylim(-1.25, 1.25)
    plt.xlabel("True Values")
    plt.ylabel("Predicted Values")
    plt.title("Prediction Accuracy")
    plt.grid(True)
    print('Done.')
    return plt


def featureCoeff(model):
    print(f'Computing feature coefficients...')
    sort_column = 'coefficient'

    coefficients = model.named_steps['regressor'].coef_.flatten()
    feature_names, clean_names = get_feature_names(model)

    coef_df = pd.DataFrame({
        'feature': clean_names,
        'raw_feature': feature_names,
        'coefficient': coefficients,
        'abs_coefficient': np.abs(coefficients)
    }).sort_values(by='coefficient', ascending=False)

    top_features = coef_df[coef_df['coefficient'] > 0].sort_values(by='coefficient', ascending=False).head(TOP_N_FEATURES)
    print('Done.')
    return top_features


def plotFeatureCoeff(top_features, n_features):
    print(f'Plotting feature coefficients...')
    plt.figure(figsize=(12, 18))
    bars = plt.barh(top_features['feature'][:n_features], top_features['coefficient'][:n_features])
    plt.xlabel('Coefficient Value')
    plt.xticks(fontsize=7)
    plt.yticks(fontsize=7)
    plt.title(f'Top {TOP_N_FEATURES} Most Influential Features (Linear Regression)')
    plt.axvline(x=0, color='gray', linestyle='--')
    plt.grid(True, axis='x', linestyle=':', alpha=0.7)
    plt.gca().invert_yaxis()
    plt.tight_layout()
    print('Done.')
    return plt


def get_feature_names(model):
    print("Getting feature names...")

    preprocessor = model.named_steps['preprocessor']
    feature_names = preprocessor.get_feature_names_out()

    # Store both prefixed and cleaned names
    cleaned_names = [name.split('__', 1)[-1] for name in feature_names]

    print(f"Done. Retrieved {len(feature_names)} feature names.")
    return feature_names, cleaned_names


def mergeTopFeatures(top_features_lists):
    print(f'Creating common top features using clean names...')
    from itertools import chain

    # Make clean names
    normalized_lists = []
    for item in top_features_lists:
        if isinstance(item, list):
            normalized_lists.append(item)
        elif hasattr(item, 'columns') and 'feature' in item.columns:
            normalized_lists.append(item['feature'].tolist())
        else:
            raise ValueError("Each item must be a list or a DataFrame with a 'feature' column")

    # Find intersection
    common_features = set(normalized_lists[0])
    for feature_list in normalized_lists[1:]:
        common_features.intersection_update(feature_list)

    # Preserve order
    seen = set()
    merged_common_ordered = []

    for item in chain.from_iterable(normalized_lists):
        if item in common_features and item not in seen:
            seen.add(item)
            merged_common_ordered.append(item)

    print('Done.')
    return merged_common_ordered


def one_hot_encode_selected(df, columns_to_encode):
    df = df.copy()
    if not columns_to_encode:
        return df
    encoded = pd.get_dummies(df[columns_to_encode], prefix=columns_to_encode)
    df = df.drop(columns=columns_to_encode)
    return pd.concat([df, encoded], axis=1)

#### Execution

In [ ]:
OFFICES = ['U.S. House', 'State House']
YEARS = ['2018', '2020', '2022', '2024']

TOP_N_FEATURES = 250

census_datasets = [
    'b02001_race', 'b04007_ancestry', 'b05012_nativity_us', 'b08303_travel_time_work', 'b25003_housing_rentership', 
    'dp02_selected_social_characteristics', 'dp03_selected_economic_characteristics', 'dp04_housing_characteristics', 'dp05_age_race', 
    's0101_age_sex', 's1101_households_families', 's1201_marital_status', 's1501_educational_attainment', 's1701_income_poverty', 
    's1903_median_income', 's2101_veteran_status', 's2201_food_stamps', 's2301_employment_status', 's2401_occupation_sex', 
    's2403_industry_sex', 's2501_occupancy_characteristics', 's2701_health_insurance', 's2503_financial_characteristics',
]

#### Feature Analysis

In [ ]:
DROP_FEATURES = True

drop_features = [
    'office_code', 'dem_share_prev', 'rep_share_prev', 'oth_share_prev', 
    'dem_share_change_prev', 'rep_share_change_prev', 'oth_share_change_prev', 
    'dem_votes_change_prev', 'rep_votes_change_prev', 'oth_votes_change_prev', 
    'registered_voters_change_prev', 'turnout_pct_change_prev', 
    'partisan_temp_prev', 'partisan_temp_change_prev', 
    'partisanship_lean_prev', 'partisanship_lean_change_prev', 'partisanship_lean_change_amount_prev' 
]

In [ ]:
df_datasets = makeDatasets(YEARS, OFFICES)

top_features_lists = []

for year in YEARS:
    print(f'Processing year {year}...')
    
    for office in OFFICES:
        print(f'Processing office {office}...')
        
        df = df_datasets[year][office].copy()
        X, y = makeFeaturesTargets(df)

        if DROP_FEATURES:
            X = X.drop(columns=drop_features)

        print(f"Training over {len(X.columns)} features...")

        model, X_train, X_test, y_train, y_test, numeric_cols = fitModel(X, y)
        y_pred = makePredictions(X_test, model)
        
        mse = mean_squared_error(y_test, y_pred)
        print("Mean Squared Error:", mse)

        r2 = model.score(X_test, y_test)
        print("R2 Score:", r2)

        plt = plotAccuracy(y_test, y_pred)
        plt.show()
        
        top_features = featureCoeff(model)
        # print(f'Top 20 features: {top_features[:50]}')

        top_features_lists.append(top_features['feature'].tolist())
        
        plt = plotFeatureCoeff(top_features, 50)
        plt.show()

#### Use only top_n features
Train from scratch on the top features detected from above steps

In [ ]:
# Merge top features from cycles
top_features_all = mergeTopFeatures(top_features_lists)
print(f'Num top features: {len(top_features_all)}')

print(f'Top feature count: {len(top_features_all)}')
print(f'Top 10 features: {top_features_all}')

#### Aggregate Historical Training
Do we have data leakage here? Maybe not, if we train on historical data and
<br>run a separate test on newer data in a following step. Say, here we
train 2018-2022, <br>and then in another cell test 2024 on the same model

In [ ]:
print(df_datasets.keys())

df_2024 = df_datasets['2024']['US_House']
del df_datasets['2024']

print(df_datasets.keys())

In [ ]:
# Later in the notebook we pull 2024 data, which does not include the columns
# below for 2018, 2020, and 2022. If these columns are left in, errors are
# thrown when processing 2024 data. This must be done at this step, since
# the model is being trained here.

# When using ALL features, some columns are not present in 2024 and must be removed
# missing_2024_cols = ['S1101_C03_014E', 'S1101_C02_014E', 'S1101_C04_014E']

YEARS = ['2018', '2020', '2022']

df_datasets = makeDatasets(YEARS, OFFICES)

dfs = []
for year in YEARS:
    print(f'Processing year {year}...')
    for office in OFFICES:
        print(f'Processing office {office}...')
        dfs.append(df_datasets[year][office].copy())
        
df = pd.concat(dfs, axis=0, ignore_index=True)
X, y = makeFeaturesTargets(df)

if DROP_FEATURES:
    X = X.drop(columns=drop_features)

# Get the correct, clean feature names.
feature_names, clean_names = get_feature_names(model)
# top_feature_mask = [name in top_features_all for name in clean_names]

# Scale X
X = X.replace({pd.NA: np.nan})
X_transformed = model.named_steps['preprocessor'].transform(X)
X_transformed_df = pd.DataFrame(X_transformed, columns=clean_names)
# X = X_transformed_df[top_features_all]

model, X_train, X_test, y_train, y_test, numeric_cols = fitModel(X, y)
y_pred = makePredictions(X_test, model)

expected_columns = X.columns.tolist()

mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

r2 = model.score(X_test, y_test)
print("R2 Score:", r2)

plt = plotAccuracy(y_test, y_pred)
plt.show()

top_features = featureCoeff(model)
print(f'Top 10 features: {top_features[:10]}')

plt = plotFeatureCoeff(top_features, 50)
plt.show()

#### Individual Cycle Future Prediction
This functionality requires objects and variables initiatlized in previous cells, including <code>model</code>
<br>and <code>numeric_cols</code>. When using aggregated cycle data, the model should be trained on historical
<br>elections only, previous to the upcoming election that is being predicted.

In [ ]:
X_2024, y_2024 = makeFeaturesTargets(df_2024)

if DROP_FEATURES:
    X_2024 = X_2024.drop(columns=drop_features)

# Expected columns from training
for col in expected_columns:
    if col not in X_2024.columns:
        X_2024[col] = np.nan  # or 0

# Drop columns from 2024 not expected by model
X_2024 = X_2024[expected_columns]

# NaN missing cells
X_2024 = X_2024.replace({pd.NA: np.nan})

# Transform/Scale
X_2024_transformed = model.named_steps['preprocessor'].transform(X_2024)

# Predict
y_pred_2024 = model.named_steps['regressor'].predict(X_2024_transformed)

# MSE
mse = mean_squared_error(y_2024, y_pred_2024)
print("Mean Squared Error:", mse)

# R2
r2 = model.score(X_2024, y_2024)
print("R2 Score:", r2)

# Accuracy
plt = plotAccuracy(y_2024, y_pred_2024)

# Top Features
top_features = featureCoeff(model)
print(f'Top 10 features: {top_features[:10]}')

plt = plotFeatureCoeff(top_features, 50)
plt.show()